# Connect to Google Drive and navigate to folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
%cd /content/drive/MyDrive/deeplify

# Install dependencies

In [ ]:
!pip install wandb
!wandb login ac108e79c3c8e424b4cb713b6c9c32aa83ce33cc

# Load data to VM

In [ ]:
!unzip chest_xray.zip -d /content/dataset

# Train models

## ResNet18 Adam

* weights name: Resnet-Adam(0.001)-batch(16).pth
* model: ResNet18
* optimizer: Adam
* learning rate: 0.001
* batch size: 16
* weighted loss not yet applied

In [ ]:
from train import train_model
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn import Conv2d, Linear
import torch
from torchvision.models import resnet18, ResNet18_Weights
from utils import mount_dataset

# Setup model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
n_classes = 2
model.conv1 = Conv2d(1, 64, kernel_size=(7,7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = Linear(in_features=512, out_features=n_classes, bias=True)
device = torch.device('cuda')
model.to(device, dtype=torch.float)

# Setup data
classes, dataloaders = mount_dataset('/content/dataset/chest_xray', batch_size=16, num_workers=2)

# Other training elements
optimizer = Adam(model.parameters(), lr=0.001, foreach=True)
criterion = CrossEntropyLoss()

model = train_model(
    model=model,
    dataloaders=dataloaders,
    criterion=criterion,
    optimizer=optimizer,
    classes=classes,
    device=device,
    num_epochs=30
    )

torch.save(model.state_dict(), 'Resnet-Adam(0.001)-batch(16).pth')

## ResNet18 SGD

* weights name: ResNet18-SGD(0.001)-batch(32).pth
* model: ResNet18
* optimizer: SGD
* learning rate: 0.001
* batch size: 32
* weighted loss not yet applied

In [ ]:
from train import train_model
from torch.nn import CrossEntropyLoss
from torch.nn import Conv2d
import torch.optim as optim
from torch.optim import lr_scheduler
import torch
from torchvision.models import resnet18, ResNet18_Weights
from utils import mount_dataset
from eval import evaluate

# Setup model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.conv1 = Conv2d(1, 64, kernel_size=(7,7), stride=(2, 2), padding=(3, 3), bias=False)
n_classes = 2
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, n_classes, bias=True)

device = torch.device('cuda')
model.to(device, dtype=torch.float)

# Setup data
classes, dataloaders = mount_dataset('/content/dataset/chest_xray', batch_size=32, num_workers=2)

# Other training elements
criterion = CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(
    model=model,
    dataloaders=dataloaders,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    classes=classes,
    device=device,
    num_epochs=20
    )

torch.save(model.state_dict(), 'ResNet18-SGD(0.001)-batch(32).pth')

evaluate(model, dataloaders['test'], device, classes)

## ResNet18 Weighted Loss

* weights name: ResNet18-SGD(0.001)-batch(32)-weightedloss.pth
* model: ResNet18
* optimizer: SGD
* learning rate: 0.001
* batch size: 32
* weighted loss applied

In [ ]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn import Conv2d, Linear
import torch.optim as optim
from torch.optim import lr_scheduler
import torch
import numpy as np
import wandb
from torchvision.models import resnet18, ResNet18_Weights
from train import train_model
from utils import mount_dataset
from eval import evaluate

# Setup model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model.conv1 = Conv2d(1, 64, kernel_size=(7,7), stride=(2, 2), padding=(3, 3), bias=False)
n_classes = 2
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, n_classes, bias=False)

device = torch.device('cuda')
model.to(device, dtype=torch.float)

# Setup data
classes, dataloaders = mount_dataset('/content/dataset/chest_xray', batch_size=32, num_workers=2)

# Other training elements
classes = ['NORMAL', 'PNEUMONIA']
data_distribution = np.array([{'NORMAL': 1341, 'PNEUMONIA': 3875}[cls] for cls in classes], dtype=float)
weights = np.linalg.norm(data_distribution)/data_distribution
weights /= weights.sum()
criterion = CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(
    model=model,
    dataloaders=dataloaders,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    classes=classes,
    device=device,
    num_epochs=20
    )

torch.save(model.state_dict(), 'ResNet18-SGD(0.001)-batch(32)-weightedloss.pth')

evaluate(model, dataloaders['test'], device, classes)

## Vision Transformer

* weights name: ViTb16-SGD(0.001)-batch(4)-weightedloss.pth
* model: ViTb16
* optimizer: SGD
* learning rate: 0.001
* batch size: 4
* weighted loss applied

In [ ]:
from torch.optim import Adam, SGD, lr_scheduler
from torch.nn import CrossEntropyLoss
from torch.nn import Conv2d, Linear
import torch, math
import numpy as np
from torchvision.models import vit_b_16, ViT_B_16_Weights
from utils import mount_dataset
from train import train_model
from eval import evaluate

image_size = 960/2
n_classes = 2

# Setup model
model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)
model.image_size = image_size
n_feat = model.conv_proj.out_channels
n_embed = int(image_size/model.patch_size)

# Adjust model to data
# Interpolate positional embedding
embedding = model.encoder.pos_embedding
tokens, image_sectors = embedding[:, :1], embedding[:, 1:]
n_sectors = int(math.sqrt(image_sectors.shape[1]))
image_sectors = image_sectors.reshape(1, n_sectors, n_sectors, -1).permute(0, 3, 1, 2)  # [1, 196, dim]->[1, 14, 14, dim]->[1, dim, 14, 14]
image_sectors = torch.nn.functional.interpolate(image_sectors, size=(n_embed, n_embed), mode='bicubic') # [1, dim, 14, 14] -> [1, dim, 24, 24]
image_sectors = image_sectors.permute(0, 2, 3, 1).reshape(1, n_embed * n_embed, -1)   # [1, dim, 24, 24] -> [1, 24*24, dim]
interpolated_embedding = torch.cat([tokens, image_sectors], dim=1)   # [1, 24*24+1, dim]
model.encoder.pos_embedding = torch.nn.Parameter(interpolated_embedding, True)
# Adjust channels and outputs
model.conv_proj = Conv2d(1, n_feat, kernel_size=(16, 16), stride=(16, 16))
model.heads.head = Linear(in_features=n_feat, out_features=n_classes, bias=False)
device = torch.device('cuda')
model = model.to(device)

# Setup data
classes, dataloaders = mount_dataset('/content/dataset/chest_xray', batch_size=16, num_workers=2, image_size=(image_size,image_size), image_resize_scale=(490, 670))

# Other training elements
data_distribution = np.array([{'NORMAL': 1341, 'PNEUMONIA': 3875}[cls] for cls in classes], dtype=float)
weights = np.linalg.norm(data_distribution)/data_distribution
weights /= weights.sum()
weights = torch.from_numpy(weights)
weights = weights.to(device, dtype=torch.float)
criterion = CrossEntropyLoss(weight=weights)
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(
    model=model,
    dataloaders=dataloaders,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    classes=classes,
    device=device,
    num_epochs=20
    )

torch.save(model.state_dict(), 'ViTb16-SGD(0.001)-batch(4)-weightedloss.pth')

evaluate(model, dataloaders['test'], device, classes)

## EfficientNet v2

* weights name: EfficientNetv2-SGD(0.001)-batch(4)-weightedloss.pth
* model: EfficientNet_v2_s
* optimizer: SGD
* learning rate: 0.001
* batch size: 4
* weighted loss applied

In [ ]:
from torch.nn import CrossEntropyLoss
from torch.nn import Conv2d
import torch.optim as optim
from torch.optim import lr_scheduler
import torch
import numpy as np
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
from train import train_model
from utils import mount_dataset
from eval import evaluate


# Setup model
model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
model.features[0][0] = Conv2d(1, 24, kernel_size=(3,3), stride=(2, 2), padding=(1,1), bias=False)
n_classes = 2
num_ftrs = model.classifier[1].in_features
model.classifier[1] = torch.nn.Linear(num_ftrs, n_classes, bias=False)

device = torch.device('cuda')
model.to(device, dtype=torch.float)

# Setup data
image_size = 960/2
classes, dataloaders = mount_dataset('/content/dataset/chest_xray', batch_size=4, num_workers=2, image_size=(image_size,image_size), image_resize_scale=(490, 670))

# Other training elements
data_distribution = np.array([{'NORMAL': 1341, 'PNEUMONIA': 3875}[cls] for cls in classes], dtype=float)
weights = np.linalg.norm(data_distribution)/data_distribution
weights /= weights.sum()
weights = torch.from_numpy(weights)
weights = weights.to(device, dtype=torch.float)
criterion = CrossEntropyLoss(weight=weights)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(
    model=model,
    dataloaders=dataloaders,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    classes=classes,
    device=device,
    num_epochs=20
    )

torch.save(model.state_dict(), 'EfficientNetv2-SGD(0.001)-batch(4)-weightedloss.pth')

evaluate(model, dataloaders['test'], device, classes)

## Swin v2

* weights name: Swinv2-SGD(0.001)-batch(8).pth
* model: Swin_v2_s
* optimizer: SGD
* learning rate: 0.001
* batch size: 8
* weighted loss applied

In [ ]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn import Conv2d, Linear
import torch.optim as optim
from torch.optim import lr_scheduler
import torch
import numpy as np
from torchvision.models import swin_v2_s, Swin_V2_S_Weights
from train import train_model
from utils import mount_dataset
from eval import evaluate

# Setup model
model = swin_v2_s(weights=Swin_V2_S_Weights.IMAGENET1K_V1)
model.features[0][0] = Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
model.head = Linear(in_features=model.head.in_features, out_features=2, bias=True)

device = torch.device('cuda')
model.to(device, dtype=torch.float)

# Setup data
image_size = 960/2
classes, dataloaders = mount_dataset('/content/dataset/chest_xray', batch_size=8, num_workers=2, image_size=(image_size,image_size), image_resize_scale=(490, 670))

# Other training elements
data_distribution = np.array([{'NORMAL': 1341, 'PNEUMONIA': 3875}[cls] for cls in classes], dtype=float)
weights = np.linalg.norm(data_distribution)/data_distribution
weights /= weights.sum()
weights = torch.from_numpy(weights)
weights = weights.to(device, dtype=torch.float)
criterion = CrossEntropyLoss(weight=weights)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(
    model=model,
    dataloaders=dataloaders,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    classes=classes,
    device=device,
    num_epochs=20
    )

torch.save(model.state_dict(), 'Swinv2-SGD(0.001)-batch(8).pth')

evaluate(model, dataloaders['test'], device, classes)